In [5]:
import pandas as pd
import numpy as np

In [6]:
df=pd.read_csv("./algo_data/vol_surfaces2.csv")
df['minute'] = pd.to_datetime(df['minute'])
df['minute'].apply(lambda x: x.tz).unique()
#for each row find 16:17:00 and compute years to maturity where maturity is 16:17:00 for each row

def get_years_to_maturity(row):
    maturity = pd.Timestamp(row['minute'].date(), tz=row['minute'].tz) + pd.Timedelta(hours=16, minutes=17)
    return (maturity - row['minute']).seconds / (365.25 * 24 * 60 * 60)

df['years_to_maturity'] = df.apply(get_years_to_maturity, axis=1)
df.loc[df['implied_spot'] == 0, ['implied_spot', 'atm_vol', 'slope', 'quadratic_term', 'scaled_slope', 'scaled_quadratic']] = np.nan

# Forward fill the NaN values
df=df.ffill().infer_objects(copy=False)

/var/folders/c1/dv7w7b2x49j3spqsxnz6y_sw0000gn/T/ipykernel_56026/3010810036.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['minute'] = pd.to_datetime(df['minute'])


In [ ]:

def apply_quadratic_volatility_model(strikes, spot, atm_vol, slope, quadratic_term, texp_years):
    """
    Apply the quadratic volatility model to new data points.
    
    Parameters:
        strikes (array-like): Array of strike prices.
        spot (float): Spot price.
        atm_vol (float): At-the-money volatility.
        slope (float): Slope of the linear term.
        quadratic_term (float): Coefficient of the quadratic term.
        texp_years (float): Time to expiration in years.
    
    Returns:
        array-like: Fitted volatilities for the given strikes.
    """
    #print(f"apply_quadratic_vol input sizes: strikes={strikes}, spot={len(spot)}, atm_vol={len(atm_vol)}, slope={len(slope)}, quadratic_term={len(quadratic_term)}, texp_years={len(texp_years)}")
    log_strikes = np.log(strikes) - np.log(spot)
    fitted_vols = atm_vol + (slope / np.sqrt(texp_years)) * log_strikes + quadratic_term * log_strikes**2
    fitted_vols= np.clip(fitted_vols, .05,1)
    return fitted_vols


In [ ]:

import py_vollib_vectorized
from copy import deepcopy
class env:
    def __init__(self, df):
        # 📌 Market Data
        self.df = df  # Full dataset
        self.df_today = None  # Subset for the episode

        # 📌 Index Tracking
        self.global_index = 0  # Tracks position in full dataset
        self.daily_index = 0  # Tracks position within today's data
        self.start_index = 0  # Stores start index for the episode
        self.max_steps = 180  # Maximum number of steps in an episode

        # 📌 Trading Variables
        self.position = 0  # Straddle position
        self.entry_price = 0  # Price when position was opened
        self.position_open_time = None  # Time step when position was opened

        # 📌 Capital & PnL
        self.capital = 100
        self.pnl = 0
        self.position_value = 0

        # 📌 Episode State
        self.done = False
        self.current_time = None
        self.current_row = None

        # 📌 Actions & Observations
        self.action_space = [0, 1, 2]  # 0: Hold, 1: Open, 2: Close
        self.observation_space = self.df.columns
        self.debug=True


    """
    def select_todays_data(self):
        self.df_today = self.df[self.df['date'] == self.current_row['date']]
        self.df_today=deepcopy(self.df_today)

        return self.df_today

    """

    def update_time_step(self, minutes=1):
        self.global_index = min(self.global_index + minutes, len(self.df) - 1)
        self.daily_index = min(self.daily_index + minutes, len(self.df_today) - 1) 

    def get_current_time(self):
        return self.df.iloc[self.global_index]['minute']
    

    def get_current_row(self):
        return self.df.iloc[self.global_index]
    
    
    
    def reset(self):
        self.global_index = 0
        self.done = False
        self.position = 0

        # Pick a valid starting point
        self.start_index = self.pick_episode_start()
        self.global_index = self.start_index

        # Select today's data and set the daily index
        self.daily_index = self.df_today.index.get_loc(self.global_index)

        straddle_prices=self.compute_daily_atm_straddle_prices()

        self.df_today["daily_straddle_prices"]=straddle_prices
        #self.current_time = self.current_row['minute']
        self.last_pnl=0


        return self.current_row

    """
    def _get_state(self):
        state = {
            'position': self.position,
            'capital': self.capital,
            'entry_price': self.entry_price,
            'position_value': self.position_value,
            'pnl': self.pnl
        }
        return state
    """
    def _get_state(self):
        """Returns the current state of the environment as a dictionary."""
        
        current_row = self.get_current_row()  # Get the current row based on global_index
        steps_taken=self.global_index-self.start_index
        steps_remaining=self.max_steps-steps_taken
        #steps_remaining = len(self.df_today) - self.global_index  # Remaining steps in today's episode
        time_to_expiration = current_row['years_to_maturity']
        
        # Position-related details
        position_status = "Open" if self.position > 0 else "Closed"
        position_strike = self.strike if self.position > 0 else None
        current_straddle_price = self.compute_straddle_prices(position_strike) if self.position > 0 else None
        spot_vol = self.compute_spot_vols(position_strike) if self.position > 0 else None
        
        # Market data
        state = {
            "current_time": current_row['minute'],  # Current datetime
            "episode_step": steps_taken,  # How far we are in the episode
            "steps_remaining": steps_remaining,  # Remaining steps
            "time_to_expiration": time_to_expiration,  # Time to expiration

            # Market & volatility data
            "current_spot": current_row['implied_spot'],
            "current_atm_vol": current_row['atm_vol'],
            "current_scaled_slope": current_row['scaled_slope'],  # Scaled slope
            "current_scaled_quadratic": current_row['scaled_quadratic'],  # Scaled quadratic
            #"spot_vol_for_straddle": spot_vol,  # Spot vol specifically for the straddle

            
            # Position & trading details
            "position_status": position_status,
            "position_strike": position_strike,
            "current_straddle_price": self.df_today["daily_straddle_prices"].loc[(self.global_index)],

            # PnL Tracking
            "cumulative_pnl": self.df_today["open_straddle_pnl"].loc[self.global_index],
            "last_step_pnl": self.df_today["open_straddle_pnl"].loc[self.global_index]-self.df_today["open_straddle_pnl"].loc[self.global_index-1] if self.global_index>0 else 0
            }
        
        return state 
    
    def pick_random_day(self, burn_days=5):
        all_days = self.df['date'].unique()
        all_days = sorted(all_days)
        start_day = np.random.choice(all_days[burn_days:-1])
        return start_day

    def pick_random_timestep(self,df):
        all_times = self.df['minute'].apply(lambda x: x.time()).unique()
        all_times = sorted(all_times)
        latest_time = pd.Timestamp('12:45').time()
        earliest_time = pd.Timestamp('9:30').time()
        all_times = [x for x in all_times if x >= earliest_time and x <= latest_time]
        start_time = np.random.choice(all_times)
        return start_time

    def pick_episode_start(self):
        start_day = self.pick_random_day()
        self.df_today = self.df[self.df['date'] == start_day]
        self.df_today=deepcopy(self.df_today)
        start_time=self.pick_random_timestep(df)
        episode_start_index = self.df[(self.df['date'] == start_day) & (self.df['minute'].apply(lambda x: x.time()) == start_time)].index[0]
        
        #self.current_row = self.df.iloc[self.global_index]
        #self.df_today=self.select_todays_data()
        return episode_start_index
    
    

    def price_one_day_straddle(self, texp,atm_vol):
        call_price = py_vollib_vectorized.models.vectorized_black_scholes('c', 100, 100, texp,0, atm_vol)/100

        put_price = py_vollib_vectorized.models.vectorized_black_scholes('p', 100, 100, texp,0, atm_vol)/100
        return call_price + put_price
    import numpy as np
    
    def compute_spot_vols(self,strike):
        """
        Compute fitted volatilities for a range of strikes.
        
        Parameters:
            spot (float): Spot price.
            atm_vol (float): At-the-money
            slope (float): Slope of the linear term.
            quadratic_term (float): Coefficient of the quadratic term.
            texp_years (float): Time to expiration in years.    

        Returns:
            array-like: Fitted volatilities for a range of strikes.
        """
        spots=self.df_today['implied_spot']
        atm_vol=self.df_today['atm_vol']
        texp_years = self.df_today['years_to_maturity']
        slope=self.df_today['slope']
        quadratic_term=self.df_today['quadratic_term']
        #print(f"variable sizes: texp={texp_years.shape}, spot={spots.shape}, atm_vol={atm_vol.shape}, slope={slope.shape}, quadratic_term={quadratic_term.shape},strike={strike.shape}")
        vols = apply_quadratic_volatility_model(strike, spots, atm_vol, slope, quadratic_term, texp_years)
        #print(f"vols size={vols.shape}")
        return vols


    def compute_daily_atm_straddle_prices(self):
        """
        Compute straddle prices for a range of strikes.
        
        Parameters:
            spot (float): Spot price.
            atm_vol (float): At-the-money
            slope (float): Slope of the linear term.
            quadratic_term (float): Coefficient of the quadratic term.
            texp_years (float): Time to expiration in years.    

        Returns:
            array-like: Fitted volatilities for a range of strikes.
        """
        texp = self.df_today['years_to_maturity']
        spot = self.df_today['implied_spot']
        texp = self.df_today['years_to_maturity']
        vol=self.df_today['atm_vol']
        #print("variable sizes: ",texp.shape,spot.shape,vol.shape)
        straddle_prices = self.price_instrument('c', spot, spot, texp, vol) + self.price_instrument('p', spot, spot, texp, vol)

        return straddle_prices

    
    def compute_straddle_prices(self, strike):
        """
        Compute straddle prices for a range of strikes.
        
        Parameters:
            spot (float): Spot price.
            atm_vol (float): At-the-money
            slope (float): Slope of the linear term.
            quadratic_term (float): Coefficient of the quadratic term.
            texp_years (float): Time to expiration in years.    

        Returns:
            array-like: Fitted volatilities for a range of strikes.
        """
    
        texp = self.df_today['years_to_maturity']
        spot = self.df_today['implied_spot']
        vols=self.compute_spot_vols(strike)
        #print(f"variable sizes: texp={texp.shape}, spot={spot.shape}, vols={vols.shape}")
        #vols=apply_apply_quadratic_volatility_model(strike, spot, atm_vols, slopes, quadratic_terms, texp)
        straddle_prices = self.price_instrument('c', strike, spot, texp, vols) + self.price_instrument('p', strike, spot, texp, vols) 
        #print(f"straddle_prices={straddle_prices}")
        return straddle_prices

    
    def update_time_step(self, minutes=1):
        self.global_index = min(self.global_index + minutes, self.df_today.index.max())


    def step(self, action):
        """
        Execute the selected action in the environment.
        """
        done = False
        reward = 0.0

        if action == 1:  # Open position
            if self.position == 0:  # Only open if no position
                self.open_position()
                self.position_open_time = self.global_index
                #print(f"current index={self.global_index}")
                self.update_time_step(60)
                #print(f"adjusted index={self.global_index}")

        elif action == 2:  # Close position
            if self.position > 0 and (self.global_index - self.position_open_time >= 60):
                pnl=self.df_today["open_straddle_pnl"].loc[self.global_index]
                reward=pnl-self.last_pnl
                self.last_pnl=pnl
                done = True  # End episode

        elif action == 3:  # Hold
            if self.position > 0:
                pnl=self.df_today["open_straddle_pnl"].loc[self.global_index]
                reward=pnl-self.last_pnl
                self.last_pnl=pnl
            self.update_time_step(1)
            if self.global_index - self.start_index >= 180:
                done=True
            if self.position > 0 and self.global_index - self.position_open_time >= 120:
                done=True

        # Get next state
        next_state = self._get_state()
        return next_state, reward, done, {}
    
    def valid_actionsa(self):
        if self.position == 0:
            return [1, 3]
        else:
            return [2, 3]

    def price_instrument(self, cp, strike, spot, texp, vol):
        #if self.debug:
        #    print(f"cp={cp}\n, strike={strike}\n, spot={spot}\n, texp={texp}\n, vol={vol}\n")
        #print(f"pricing_insturment sizes: cp={cp}, strike={strike.shape}, spot={spot.shape}, texp={texp.shape}, vol={vol.shape}")
        return py_vollib_vectorized.models.vectorized_black_scholes(cp, spot, strike, texp, 0, vol,return_as="numpy")



    def open_position(self):

        ivol = self.get_current_row()['implied_spot']
        texp = self.get_current_row()['years_to_maturity']
        spot=self.get_current_row()['implied_spot']
        #straddle_price_1 = self.price_one_day_straddle(texp, ivol)
        straddle_price=self.df_today['daily_straddle_prices'].loc[self.global_index]
        #print(f"straddle_price={straddle_price}")
        #print(f"straddle_price_1={straddle_price_1}")
        self.position = self.capital / straddle_price
        #self.position_value = self.position * straddle_price
        self.strike=spot
        #spot_vols=self.compute_spot_vols(self.strike)
        self.straddle_prices=self.compute_straddle_prices(self.strike)
        self.df_today["open_straddle_prices"]=self.straddle_prices
        self.df_today["open_straddle_pnl"]=(self.df_today["open_straddle_prices"]- straddle_price)*self.position
        
        self.position_open_time = self.global_index
        return self.position_value


env = env(df)
env.reset()
#env.current_time[0]
for i in range(2):  
    row=env.pick_episode_start()
    current_time = env.get_current_time()
    if current_time.time() < pd.Timestamp('9:30').time():
        print('error')
    if current_time.time() > pd.Timestamp('12:45').time():
        print('error')
env.reset()
#env.price_one_day_straddle(0.5, 0.2)
#pos_value = env.open_position(1)
#env.position_value
env.df_today
strike=env.df_today['implied_spot'].iloc[0]
vols=env.compute_spot_vols(strike)
#straddle_prices=env.compute_straddle_prices(strike,vols)
from copy import deepcopy
df_episode=deepcopy(env.df_today)

df_episode.reset_index(inplace=True)
env.global_index
env.df_today
env.global_index

#env.open_position()
#print (f"current index={env.global_index}")
(s, reward, done,bla)=env.step(1)
print(f"s={s}")
(s,reward,done,bla)=env.step(3)
print(f"s={s}")
(s,reward,done,bla)=env.step(3)
print(f"s={s}")
(s,reward,done,bla)=env.step(2)
print(f"s={s}")
#print (f"current index={env.global_index}")

s={'current_time': Timestamp('2025-01-08 11:55:00-0500', tz='UTC-05:00'), 'episode_step': np.int64(60), 'steps_remaining': np.int64(120), 'time_to_expiration': np.float64(0.000498136740436535), 'current_spot': np.float64(589.2243119635193), 'current_atm_vol': np.float64(0.2570782451158043), 'current_scaled_slope': np.float64(-0.0058665231613695), 'current_scaled_quadratic': np.float64(0.1569409447599382), 'position_status': 'Open', 'position_strike': np.float64(588.1735592329919), 'current_straddle_price': np.float64(2.69748934339036), 'cumulative_pnl': np.float64(54.68223630644037), 'last_step_pnl': np.float64(-8.44029588003935)}
s={'current_time': Timestamp('2025-01-08 11:56:00-0500', tz='UTC-05:00'), 'episode_step': np.int64(61), 'steps_remaining': np.int64(119), 'time_to_expiration': np.float64(0.0004962354551676934), 'current_spot': np.float64(589.1673398804442), 'current_atm_vol': np.float64(0.2584174761497063), 'current_scaled_slope': np.float64(-0.0056401645488066), 'current_sc

In [75]:
env.global_index
env.start_index
env.position

env.df_today[["minute","open_straddle_prices","implied_spot","open_straddle_pnl","atm_vol","years_to_maturity"]].loc[env.global_index+180-5:env.global_index+180+5]


,minute,open_straddle_prices,implied_spot,open_straddle_pnl,atm_vol,years_to_maturity
99244,2025-01-08 14:52:00-05:00,3.026943,588.695817,-5.993778,0.278835,0.000162
99245,2025-01-08 14:53:00-05:00,2.650729,588.532076,-17.677671,0.276319,0.000160
99246,2025-01-08 14:54:00-05:00,2.559700,588.467016,-20.504688,0.281709,0.000158
99247,2025-01-08 14:55:00-05:00,1.969170,588.280063,-38.844483,0.288890,0.000156
99248,2025-01-08 14:56:00-05:00,2.643580,588.484900,-17.899697,0.284731,0.000154
99249,2025-01-08 14:57:00-05:00,3.053424,588.585338,-5.171365,0.281580,0.000152
99250,2025-01-08 14:58:00-05:00,3.392762,588.718805,5.367270,0.277012,0.000150
99251,2025-01-08 14:59:00-05:00,2.873232,588.532403,-10.767496,0.282577,0.000148
99252,2025-01-08 15:00:00-05:00,3.115769,588.527434,-3.235139,0.280964,0.000146
99253,2025-01-08 15:01:00-05:00,3.381244,588.855325,5.009567,0.272407,0.000144


In [68]:
env.df_today[["minute","open_straddle_prices","implied_spot","open_straddle_pnl","atm_vol","years_to_maturity"]].loc[env.start_index-5:env.start_index+5]

,minute,open_straddle_prices,implied_spot,open_straddle_pnl,atm_vol,years_to_maturity
99002,2025-01-08 10:50:00-05:00,1.353124,587.391553,-57.976716,0.282635,0.000622
99003,2025-01-08 10:51:00-05:00,2.453787,587.852097,-23.793981,0.276480,0.000620
99004,2025-01-08 10:52:00-05:00,3.128082,588.147393,-2.852765,0.273663,0.000618
99005,2025-01-08 10:53:00-05:00,3.855132,588.435048,19.726857,0.275689,0.000616
99006,2025-01-08 10:54:00-05:00,2.662566,587.931025,-17.310044,0.277830,0.000614
99007,2025-01-08 10:55:00-05:00,3.219939,588.173559,0.000000,0.277301,0.000612
99008,2025-01-08 10:56:00-05:00,3.228943,588.182664,0.279650,0.276554,0.000610
99009,2025-01-08 10:57:00-05:00,2.885932,588.036596,-10.373072,0.276696,0.000608
99010,2025-01-08 10:58:00-05:00,3.423228,588.271334,6.313438,0.275462,0.000607
99011,2025-01-08 10:59:00-05:00,3.318560,588.225939,3.062828,0.276668,0.000605


In [78]:
env.compute_straddle_prices(env.strike)

array([3.71553894, 4.29794691, 3.16831453, 2.90759561, 4.21250399,
       3.96706416, 4.65302103, 3.61337264, 3.15302232, 2.18331249,
       2.68837073, 2.85329106, 2.88596353, 2.8382195 , 2.94688018,
       2.77851859, 1.55607457, 2.772655  , 2.39947097, 3.565635  ,
       3.70341209, 4.35629049, 1.79867227, 1.45467966, 1.78451565,
       1.75475018, 2.12405291, 1.40484762, 1.7791483 , 1.77246044,
       2.85751599, 2.85592266, 4.53976425, 4.63192725, 4.58626124,
       4.32489158, 4.24933114, 4.06061504, 3.71918097, 3.94365248,
       1.82546003, 2.07037791, 2.08345328, 1.47831735, 1.71382399,
       1.90877153, 1.61580917, 1.64583147, 1.3219107 , 1.28940423,
       2.05159756, 1.69874233, 1.80642518, 1.35650163, 1.13591143,
       1.130873  , 1.12038877, 1.46601425, 1.71399915, 1.71309551,
       1.45647166, 1.71841727, 1.64307408, 1.31396907, 1.30910951,
       1.64513326, 2.43786164, 2.96463132, 1.9176245 , 1.19967835,
       1.36732713, 1.48016683, 1.26923006, 1.07101255, 1.05756

In [109]:
# Replace zero implied_spot with NaN to use forward fill
from copy import deepcopy
df = deepcopy(env.df_today)
# Replace zero implied_spot with NaN to use forward fill
df.loc[df['implied_spot'] == 0, ['implied_spot', 'atm_vol', 'slope', 'quadratic_term', 'scaled_slope', 'scaled_quadratic']] = np.nan

# Forward fill the NaN values
df.ffill().infer_objects(copy=False)
#df=df.fillna(method='ffill')

# Verify the changes
df

/var/folders/c1/dv7w7b2x49j3spqsxnz6y_sw0000gn/T/ipykernel_50418/1879655896.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill().infer_objects(copy=False)


,minute,implied_spot,atm_vol,slope,quadratic_term,scaled_slope,scaled_quadratic,open_price,high_price,low_price,...,volume,vwap,timestamp,transactions,otc,timestamp_utc,timestamp_est,date,years_to_maturity,daily_straddle_prices
110653,2025-02-24 09:31:00-05:00,602.144623,0.238504,-5.468350,148.229254,-0.015193,0.114421,602.0200,602.18,601.770,...,833091.0,601.9241,1740407400000,11891,NaN,2025-02-24 14:30:00+00:00,2025-02-24 09:30:00-05:00,2025-02-24,0.000772,3.183630
110654,2025-02-24 09:32:00-05:00,602.443336,0.230018,-5.530652,127.049323,-0.015347,0.097831,602.1500,602.46,602.090,...,147201.0,602.2773,1740407460000,2181,NaN,2025-02-24 14:31:00+00:00,2025-02-24 09:31:00-05:00,2025-02-24,0.000770,3.068087
110655,2025-02-24 09:33:00-05:00,602.851492,0.225284,-5.639400,118.401947,-0.015630,0.090947,602.4200,603.03,602.370,...,152520.0,602.8236,1740407520000,2245,NaN,2025-02-24 14:32:00+00:00,2025-02-24 09:32:00-05:00,2025-02-24,0.000768,3.003272
110656,2025-02-24 09:34:00-05:00,602.237656,0.232385,-5.300987,132.715588,-0.014673,0.101689,602.8255,602.85,602.170,...,135333.0,602.4473,1740407580000,1672,NaN,2025-02-24 14:33:00+00:00,2025-02-24 09:33:00-05:00,2025-02-24,0.000766,3.090940
110657,2025-02-24 09:35:00-05:00,602.154755,0.231735,-4.995026,94.329581,-0.013809,0.072098,602.2200,602.40,602.145,...,103532.0,602.2356,1740407640000,1354,NaN,2025-02-24 14:34:00+00:00,2025-02-24 09:34:00-05:00,2025-02-24,0.000764,3.078044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111039,2025-02-24 15:57:00-05:00,597.270669,0.245910,-7.344222,4384.045996,-0.004529,0.166706,597.5400,597.68,597.220,...,724533.0,598.7368,1740430560000,3531,NaN,2025-02-24 20:56:00+00:00,2025-02-24 15:56:00-05:00,2025-02-24,0.000038,0.722647
111040,2025-02-24 15:58:00-05:00,597.116676,0.249654,-3.903372,4926.766989,-0.002346,0.177977,597.2300,597.24,597.030,...,600166.0,598.2853,1740430620000,3172,NaN,2025-02-24 20:57:00+00:00,2025-02-24 15:57:00-05:00,2025-02-24,0.000036,0.714887
111041,2025-02-24 15:59:00-05:00,596.938392,0.243920,-5.145935,5369.315402,-0.003010,0.183755,597.0600,597.11,596.870,...,655443.0,596.9657,1740430680000,5358,NaN,2025-02-24 20:58:00+00:00,2025-02-24 15:58:00-05:00,2025-02-24,0.000034,0.679636
111042,2025-02-24 16:00:00-05:00,597.267837,0.228383,-7.279024,5934.642665,-0.004138,0.191819,596.9000,597.25,596.640,...,2625402.0,597.1539,1740430740000,12252,NaN,2025-02-24 20:59:00+00:00,2025-02-24 15:59:00-05:00,2025-02-24,0.000032,0.618758


In [55]:
env.df_today.loc[env.global_index]
#env.df_today.index[env.current_time[0]]
env.current_time[0]

Timestamp('2024-01-02 09:31:00-0500', tz='UTC-05:00')

In [110]:
env.daily_index
env.df_today


,minute,implied_spot,atm_vol,slope,quadratic_term,scaled_slope,scaled_quadratic,open_price,high_price,low_price,...,volume,vwap,timestamp,transactions,otc,timestamp_utc,timestamp_est,date,years_to_maturity,daily_straddle_prices
110653,2025-02-24 09:31:00-05:00,602.144623,0.238504,-5.468350,148.229254,-0.015193,0.114421,602.0200,602.18,601.770,...,833091.0,601.9241,1740407400000,11891,NaN,2025-02-24 14:30:00+00:00,2025-02-24 09:30:00-05:00,2025-02-24,0.000772,3.183630
110654,2025-02-24 09:32:00-05:00,602.443336,0.230018,-5.530652,127.049323,-0.015347,0.097831,602.1500,602.46,602.090,...,147201.0,602.2773,1740407460000,2181,NaN,2025-02-24 14:31:00+00:00,2025-02-24 09:31:00-05:00,2025-02-24,0.000770,3.068087
110655,2025-02-24 09:33:00-05:00,602.851492,0.225284,-5.639400,118.401947,-0.015630,0.090947,602.4200,603.03,602.370,...,152520.0,602.8236,1740407520000,2245,NaN,2025-02-24 14:32:00+00:00,2025-02-24 09:32:00-05:00,2025-02-24,0.000768,3.003272
110656,2025-02-24 09:34:00-05:00,602.237656,0.232385,-5.300987,132.715588,-0.014673,0.101689,602.8255,602.85,602.170,...,135333.0,602.4473,1740407580000,1672,NaN,2025-02-24 14:33:00+00:00,2025-02-24 09:33:00-05:00,2025-02-24,0.000766,3.090940
110657,2025-02-24 09:35:00-05:00,602.154755,0.231735,-4.995026,94.329581,-0.013809,0.072098,602.2200,602.40,602.145,...,103532.0,602.2356,1740407640000,1354,NaN,2025-02-24 14:34:00+00:00,2025-02-24 09:34:00-05:00,2025-02-24,0.000764,3.078044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111039,2025-02-24 15:57:00-05:00,597.270669,0.245910,-7.344222,4384.045996,-0.004529,0.166706,597.5400,597.68,597.220,...,724533.0,598.7368,1740430560000,3531,NaN,2025-02-24 20:56:00+00:00,2025-02-24 15:56:00-05:00,2025-02-24,0.000038,0.722647
111040,2025-02-24 15:58:00-05:00,597.116676,0.249654,-3.903372,4926.766989,-0.002346,0.177977,597.2300,597.24,597.030,...,600166.0,598.2853,1740430620000,3172,NaN,2025-02-24 20:57:00+00:00,2025-02-24 15:57:00-05:00,2025-02-24,0.000036,0.714887
111041,2025-02-24 15:59:00-05:00,596.938392,0.243920,-5.145935,5369.315402,-0.003010,0.183755,597.0600,597.11,596.870,...,655443.0,596.9657,1740430680000,5358,NaN,2025-02-24 20:58:00+00:00,2025-02-24 15:58:00-05:00,2025-02-24,0.000034,0.679636
111042,2025-02-24 16:00:00-05:00,597.267837,0.228383,-7.279024,5934.642665,-0.004138,0.191819,596.9000,597.25,596.640,...,2625402.0,597.1539,1740430740000,12252,NaN,2025-02-24 20:59:00+00:00,2025-02-24 15:59:00-05:00,2025-02-24,0.000032,0.618758


In [3]:
df

,minute,implied_spot,atm_vol,slope,quadratic_term,scaled_slope,scaled_quadratic,open_price,high_price,low_price,close_price,volume,vwap,timestamp,transactions,otc,timestamp_utc,timestamp_est,date
0,2024-01-02 09:31:00-05:00,472.563933,0.189797,-0.386711,133.447533,-0.001074,0.103011,472.160,472.67,472.050,472.5300,1190522.0,472.9271,1704205800000,11700,NaN,2024-01-02 14:30:00+00:00,2024-01-02 09:30:00-05:00,2024-01-02
1,2024-01-02 09:32:00-05:00,472.705855,0.183684,-0.393608,156.933023,-0.001092,0.120842,472.530,472.70,472.365,472.6812,756655.0,473.3993,1704205860000,3810,NaN,2024-01-02 14:31:00+00:00,2024-01-02 09:31:00-05:00,2024-01-02
2,2024-01-02 09:33:00-05:00,472.784149,0.181348,-0.452450,182.755226,-0.001254,0.140378,472.680,472.80,472.650,472.7800,693889.0,473.6179,1704205920000,3807,NaN,2024-01-02 14:32:00+00:00,2024-01-02 09:32:00-05:00,2024-01-02
3,2024-01-02 09:34:00-05:00,472.743159,0.180591,-0.333196,179.169754,-0.000922,0.137283,472.760,472.77,472.480,472.6900,404264.0,472.6133,1704205980000,3280,NaN,2024-01-02 14:33:00+00:00,2024-01-02 09:33:00-05:00,2024-01-02
4,2024-01-02 09:35:00-05:00,472.689797,0.178740,-0.421782,178.336217,-0.001166,0.136305,472.685,472.78,472.550,472.6700,250698.0,472.6734,1704206040000,2284,NaN,2024-01-02 14:34:00+00:00,2024-01-02 09:34:00-05:00,2024-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112994,2025-03-03 15:57:00-05:00,583.217755,0.433006,-0.643380,2250.084372,-0.000397,0.085561,583.120,583.38,583.080,583.1900,531037.0,583.2175,1741035360000,5290,NaN,2025-03-03 20:56:00+00:00,2025-03-03 15:56:00-05:00,2025-03-03
112995,2025-03-03 15:58:00-05:00,583.318760,0.399562,-1.119237,2824.657505,-0.000673,0.102039,583.220,583.42,582.800,583.3300,694988.0,583.1694,1741035420000,6733,NaN,2025-03-03 20:57:00+00:00,2025-03-03 15:57:00-05:00,2025-03-03
112996,2025-03-03 15:59:00-05:00,583.589449,0.375949,-6.439821,3647.324778,-0.003767,0.124823,583.320,583.61,583.115,583.5700,829842.0,584.9927,1741035480000,6584,NaN,2025-03-03 20:58:00+00:00,2025-03-03 15:58:00-05:00,2025-03-03
112997,2025-03-03 16:00:00-05:00,584.010249,0.336107,0.051465,3648.685396,0.000029,0.117932,583.570,584.11,583.200,583.9700,2382661.0,585.3033,1741035540000,12913,NaN,2025-03-03 20:59:00+00:00,2025-03-03 15:59:00-05:00,2025-03-03
